# Добро пожаловать! 

[**ТЕКСТОВАЯ ИНСТРУКЦИЯ**](https://drive.google.com/file/d/1LV7U5F8JFAnFdOuY6GDsTQWyb8H5s-9m/view?usp=sharing), как пользоваться Jupyter Notebook или Google Colab, если вы впервые в жизни сталкиваетесь с необходимостью работать с подобными программами.

[**ВИДЕО С ПОШАГОВОЙ ИНСТРУКЦИЕЙ**](https://youtu.be/IN2132LRLyw), как пользоваться этой программой.

# Начало. Стартовая ячейка для запуска ##

В *Ячейке №0* (ниже) собраны важные функции и библиотеки языка Python, без которых программа работать не будет.

**Запустите её перед использованием программы!**

Номер ячейки указывается вверху ячейки бирюзовым/ зелёным цветом после хэштега (#).

In [ ]:
# Ячейка №0

import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver as wb
import ipywidgets as widgets
import datetime
import winsound
duration = 1000  # milliseconds
freq = 440  # Hz
print ('Библиотеки успешно загружены! :)')

# Cайт суда. Шаг №1

Ввод базовой информации, установка драйвера

1. Запустите *Ячейку №1*. 
2. **Введите** в выпадающем поле **адрес сайта** cуда (URL, полное доменное имя с протоколом), но **БЕЗ закрывающей косой черты/** (слэша). Пример: "http://kirovsky.svd.sudrf.ru"
3. **Введите название суда**, каким вы хотите его видеть в итоговой таблице. Например, для удобства стоит написать не "Кировский районный суд города Екатеринбурга Свердловской области", а "Кировский Екатеринбург".
4. **Скачайте драйвер** (см. текст между *Ячейкой №1* и *Ячейкой №2*), если он ещё не установлен на вашем компьютере.
5. **Введите путь к драйверу** в выпадающем поле, запустив *Ячейку №2*.

In [ ]:
# Ячейка №1

court = input ('Введите сайт суда в формате "http://court-name.sudrf.ru": ')
court_name = input ('Введите название суда: ')

Сайты судов защищены от примитивного парсинга (то есть от простейшего автоматического сбора информации), поэтому коду нужно имитировать действия живого человека. Для этого нужен специальный драйвер, который будет открывать и закрывать окошки браузера, собирая с сайтов нужные данные.


**Скачать драйвер** для браузера Chrome [**можно тут**](https://sites.google.com/a/chromium.org/chromedriver/downloads).

In [ ]:
# Ячейка №2

path_to_driver = input ('Введите путь к драйверу в формате "C:/Users/User/Documents/chromedriver.exe": ')

# Даты за год. Шаг №2

Выберите период, за который вы хотите получить данные. Для этого:

1. Запустите *Ячейку №3*.
2. Выберите в виджете дату начала и дату окончания желаемого периода.
3. Запустите *Ячейку №4*.
4. Проверьте в отображённых результатах, что выбраны именно те даты, которые вам нужны.
5. Запустите *Ячейку №5*.
6. Скопируйте какую-нибудь ссылку из результатов вручную и откройте её в новом окне браузе. Так вы проверите, корректно ли сработал код.

In [ ]:
# Ячейка №3

start_date = widgets.DatePicker(
    description='Начало: ',
    disabled=False
)

end_date = widgets.DatePicker(
    description='Конец: ',
    disabled=False
)

display (start_date, end_date)

In [ ]:
# Ячейка №4

mydates = pd.date_range (start_date.value, end_date.value).tolist ()

all_dates = []

for i in mydates:
    timestampStr = i.strftime("%d.%m.%Y")
    all_dates.append (timestampStr)

all_dates

In [ ]:
# Ячейка №5

all_table_urls = []

for every_date in all_dates:
    new_table_url = court + '/modules.php?name=sud_delo&srv_num=1&H_date=' + every_date
    all_table_urls.append (new_table_url)

all_table_urls

**Иногда** бывает так, что **к одному району** относятся **несколько судов**. У каждого суда — свой сервер. Проверьте по адресу сайтов, скорее всего их **URL отличается цифрой** в части **"srv_num="**. Если это так, то определите номер сервера и впишите его при запуске ячейки ниже.

**Пример**: Орджоникидзевский районный суд г. Перми — [**посмотрите вот тут**](https://ordgonik--perm.sudrf.ru/modules.php?name=sud_delo) — подразделяется на:
* Уголовные и административные дела. (ул. Кавказкая 11) — srv_num=1
* Гражданские и административные дела (ул. Валежная 13) — srv_num=2

В адресе (URL) суда с ул. Валежной (гражданские дела) нужная часть выделена жирным: 

https:// ordgonik--perm.sudrf.ru /modules.php?name=sud_delo& **srv_num=2** &H_date=10.09.2020

То есть если Вы хотите собрать данные гражданских и административных дел с улицы Валежной, то запустите ячейку ниже (*Ячейку №6*) и введите "2".

In [ ]:
# Ячейка №6

new_serv = input ()

all_table_urls = []

for every_date in all_dates:
    new_table_url = court + f'/modules.php?name=sud_delo&srv_num={new_serv}&H_date=' + every_date
    all_table_urls.append (new_table_url)

all_table_urls

# Алгоритм. Шаг №3

1. Спарсить все таблицы за все нужные дни и ссылки с них
2. Удалить дубликаты
3. Пройтись по ссылкам и собрать даты

## Первый этап

Запустите *Ячейку №7*, чтобы пройтись по всем выбранным датам и собрать ссылки на дела, которые слушались за этот период.

**Не пугайтесь всплывающих окон**: только так код может собрать нужную информацию.

Когда код завершит работу, раздастся звуковой сигнал.

Скопируйте какую-нибудь ссылку из результатов вручную и откройте её в новом окне браузе. Так вы проверите, корректно ли сработал код.

In [ ]:
# Ячейка №7

all_extracted_links = []

all_extracted_links_with_sep = []

for every_day in all_table_urls:
    
    driver = wb.Chrome(path_to_driver)
    driver.get(every_day)
    soup_b = BeautifulSoup(driver.page_source, 'lxml')
    driver.close()
    
    try:
        soup_b_new = soup_b.find (id='tablcont').find ('tbody')#.find_all ('tr')
        all_links = soup_b_new.find_all ('a')
        delete_need = soup_b_new.find_all ('a', {'title':'Решение'})
        for a in all_links: 
            if a in delete_need:
                del (a)

        extracted_links = []

        for link in all_links:
            key = link.get('href')
            extracted_links.append (court + key)

            all_extracted_links.append (court + key)

        all_extracted_links_with_sep.append (extracted_links)
        
    except AttributeError:
        pass
    
winsound.Beep(freq, duration)

all_extracted_links

## Второй этап

Запустите *Ячейку №8*, чтобы удалить дубликаты ссылок и ссылки на тексты решений.

Все собранные **ссылки сохранятся в** текстовый **файл** "Ссылки *название суда*.txt"

In [ ]:
# Ячейка №8

print (f'Всего ссылок: {len (all_extracted_links)}')

# Без дубликатов

all_extracted_links_without_duplicates = list (set (all_extracted_links))
print (f'Всего ссылок без дубликатов: {len (all_extracted_links_without_duplicates)}')

copy_all_extracted_links_without_duplicates = all_extracted_links_without_duplicates.copy()

new_clear_list = []
for i in copy_all_extracted_links_without_duplicates:
    if 'name_op=doc' not in i:
        new_clear_list.append(i)
        
print (f'Всего ссылок без дубликатов и текстов решений: {len (new_clear_list)}')

with open(f"Ссылки {court_name}.txt", "w") as text_file:
    text_file.write(str(new_clear_list))

## Третий этап

**Сколько ссылок (дел)** вы хотите спарсить**? Укажите** это в выпадающем поле, запустив *Ячейку №9*.

**Не пугайтесь всплывающих окон**: только так код может собрать нужную информацию.

Когда код **завершит** работу, раздастся **звуковой сигнал**.

**После** этого обязательно **сохраните файл** в *Ячейке №10* **ниже**.

In [ ]:
# Ячейка №9

want_links_amount = int (input ('Сколько ссылок? ')) - 1

final_df = pd.DataFrame()

for every_link in new_clear_list [0 : want_links_amount]:
    driver = wb.Chrome(path_to_driver)
    driver.get(every_link)
    soup_for_parsing = BeautifulSoup(driver.page_source, 'lxml')
    driver.close()
    
    try:
        soup_for_parsing = soup_for_parsing.find (id='cont2').find ('tbody')
        table_content_for_parsing = []
        for i in soup_for_parsing:
            z = []
            for k in i:
                z.append(k.text)
            table_content_for_parsing.append(z)
        table_for_parsing = pd.DataFrame(table_content_for_parsing)
        table_for_parsing.columns = table_for_parsing.iloc[1]
        table_for_parsing = table_for_parsing.drop(0).drop(1)
        table_for_parsing ['Ссылка'] = every_link
        needed_columns_for_parsing = pd.DataFrame ([table_for_parsing ['Ссылка'], table_for_parsing['Дата'], 
                                                    table_for_parsing ['Дата размещения']]).T
        final_df = final_df.append (needed_columns_for_parsing)
    
    except AttributeError:
        pass

winsound.Beep(freq, duration)    
    
final_df

**Сохраните файл** на компьютер, запустив *Ячейку №10*.

Название файла будет строиться по схеме: "*Название суда*_*Количество собранных ссылок*_чистый.xlsx"

Например: Дзержинский_1000_чистый.xlsx

In [ ]:
# Ячейка №10

final_df.to_excel (f"{court_name}_{want_links_amount + 1}_чистый.xlsx")
print ('Выгрузка прошла успешно! :)')

## Четвёртый этап

1. Запустите *Ячейку №11*, чтобы **посчитать разницу** (дельту) между Датой размещения и Датой события.
2. Запустите *Ячейку №12*, чтобы **сохранить** полученную **таблицу в файл**. 
    1. Название файла будет строиться по схеме: "*Название суда* _ *Количество собранных ссылок* _ даты и дельта.xlsx" 
    2. Например: *Дзержинский*_*1000* _ даты и дельта.xlsx
3. Запустите *Ячейку №13*, чтобы **подсчитать уникальные значения дельт**. 
    1. Допустим, сколько записей суды опубликовали в день события? Иными словами, сколько раз дельта между датой размещения и датой события равна 0?
4. Запустите *Ячейку №14*, чтобы **сохранить** полученную **таблицу в файл**. 
    1. Название файла будет строиться по схеме: "*Название суда*_*Количество собранных ссылок* _с дельтой.xlsx" 
    2. Например: *Дзержинский*_*1000* _с дельтой.xlsx

In [ ]:
# Ячейка №11

final_df_2 = final_df.copy()
final_df_2 [['Дата', 'Дата размещения']] = final_df_2[['Дата', 'Дата размещения']].apply(pd.pandas.to_datetime) 
final_df_2 ['Дельта'] = final_df_2['Дата размещения'] - final_df_2['Дата']
final_df_2

In [ ]:
# Ячейка №12

final_df_2.to_excel (f"{court_name}_{want_links_amount + 1}_ даты и дельта.xlsx")
print ('Выгрузка прошла успешно! :)')

In [ ]:
# Ячейка №13

gig = pd.DataFrame(final_df_2['Дельта'].value_counts()).reset_index()
gig_str = gig['index'].astype(str)
keyj = []
for i in gig_str:
    keyj.append (i[:-24])
gig ['pure'] = keyj
gig ['pure'] = gig ['pure'].astype(int)
gig

In [ ]:
# Ячейка №14

gig.to_excel (f"{court_name}_{want_links_amount + 1}_с дельтой.xlsx")

# Сравнение

**Внимание!** Это **отдельный** этап! 

Вы можете приступить к нему, минуя шаги №1-3 и запустив лишь начальную *Ячейку №0*. Тем не менее, чтобы работать с этапом "Сравнение", у вас уже должны быть типовые файлы с данными судов, созданные в ячейках выше. 

Если у вас уже есть такие файлы, вы можете сразу переходить к этому этапу. Если нет, то сначала создайте их в ячейках выше.

**Запустите большую *Ячейку №15*.**

**Не пугайтесь** её размера: она нужна, чтобы создавать и агрегировать однотипную информацию о судах.

Затем продвигайтесь ниже, к описанию *Ячейки №16*.

In [ ]:
# Ячейка №15

class Courts:
    
    def __init__(self, name = '', pure = pd.DataFrame(), delta_count = pd.DataFrame(), 
                 zero = (), week = (), month = (), more = (), maximum = (), minimum = (), below = (), 
                 none_happened = (), none_published = (), none_all = (), all_cases = (),
                 zero_percentage = (), week_percentage = (), month_percentage = (), more_percentage = (),
                 below_percentage = (), none_happened_percentage = (), none_published_percentage = (), none_all_percentage = ()):
        
        ''' Этот класс создаёт объект "суд", то есть берёт из исходных файлов данные одного суда 
        и рассчитывает следующие показатели:

        name — название суда (вводится пользователем вручную) 

        pure — файл с припиской "_чистый.xlsx", созданный в результате работы с ячейками выше. 
        Название файла вводится пользователем вручную.

        delta_count — файл с припиской "_с дельтой.xlsx", созданный в результате работы с ячейками выше. 
        Название файла вводится пользователем вручную.

        all_cases — совокупное количество записей.

        zero — количество записей, где разница между Датой размещения и Датой события составляет 0 дней.
        zero_percentage — процент zero от всех записей (all_cases).

        week — количество записей, где разница между Датой размещения и Датой события составляет от 1 до 7 дней.
        week_percentage — процент week от всех записей (all_cases).

        month — количество записей, где разница между Датой размещения и Датой события составляет от 8 до 31 дня.
        month_percentage — процент month от всех записей (all_cases).

        more — количество записей, где разница между Датой размещения и Датой события составляет свыше 31 дня.
        more_percentage — процент more от всех записей (all_cases).

        maximum — максимальная задержка публикации, то есть максимальная разница между Датой размещения и Датой события. 

        minimum — максимальная отрицательная разница между Датой размещения и Датой события, то есть наиболее отдалённое, 
        запланированное событие.

        below — количество записей, где разница между Датой размещения и Датой события отрицательна, 
        то есть событие запланировано на будущее.
        below_percentage — процент below от всех записей (all_cases).

        none_happened — количество записей, где не указана Дата события.
        none_happened_percentage — процент none_happened от всех записей (all_cases).

        none_published — количество записей, где не указана Дата размещения.
        none_published_percentage — процент none_published от всех записей (all_cases).

        none_all — количество отсутствующих дат (сумма none_happened и none_published).
        none_all_percentage — процент none_all от всех записей (all_cases).'''
        
        name = input ('Введите название суда: ')
        input_pure = input ('Введите название файла С ДАТАМИ в формате "Мотовилихинский_1000_чистый.xlsx": ')
        input_delta_count = input ('Введите название файла С ДЕЛЬТОЙ в формате "Мотовилихинский_1000_с дельтой.xlsx": ')
        pure = pd.read_excel (input_pure).drop ('Unnamed: 0', axis = 1)
        delta_count = pd.read_excel (input_delta_count).drop ('Unnamed: 0', axis = 1)
        self.name = name
        self.pure = pure
        self.delta_count = delta_count
        self.zero = self.delta_count.iloc[0]['Дельта']
        self.week = sum (self.delta_count ['Дельта']  [(self.delta_count ['pure'] < 8) & (self.delta_count ['pure'] > 0)])
        self.month = sum (self.delta_count  ['Дельта']  [(self.delta_count ['pure'] < 32) & (self.delta_count ['pure'] > 7)])
        self.more = sum (self.delta_count ['Дельта']  [(self.delta_count ['pure'] > 31)])
        self.maximum = self.delta_count ['pure'].max()
        self.minimum = self.delta_count ['pure'].min()
        self.below = sum (self.delta_count ['Дельта']  [(self.delta_count ['pure'] < 0)])
        self.none_happened = self.pure.isna().sum()[0]
        self.none_published = self.pure.isna().sum()[1]
        self.none_all = self.none_happened + self.none_published
        self.all_cases = len (self.pure.index)
        self.zero_percentage = int (self.zero * 100 / self.all_cases)
        self.week_percentage = int (self.week * 100 / self.all_cases)
        self.month_percentage = int (self.month * 100 / self.all_cases)
        self.more_percentage = int (self.more * 100 / self.all_cases)
        self.below_percentage = int (self.below * 100 / self.all_cases)
        self.none_happened_percentage = int (self.none_happened * 100 / self.all_cases)
        self.none_published_percentage = int (self.none_published * 100 / self.all_cases)
        self.none_all_percentage = int (self.none_all * 100 / self.all_cases)
    
    def create_a_row (self):
        
        ''' Создаёт список из полученных в ходе инициализации переменных для добавления в итоговый датафрейм. '''
        
        return [self.name, self.zero, self.week, self.month, self.more, self.maximum, self.minimum, 
                self.below, self.none_happened, self.none_published, self.none_all, self.all_cases,
               self.zero_percentage, self.week_percentage, self.month_percentage, self.more_percentage,
               self.below_percentage, self.none_happened_percentage, self.none_published_percentage, self.none_all_percentage]

**Запустите *Ячейку №16*,** чтобы создать таблицу и занести в неё первый ряд с данными.

В **выпадающих полях** впишите:
1. Название суда
2. Название файла с окончанием "... _ чистый.xlsx".
3. Название файла с окончанием "... _ с дельтой.xlsx".

Следуйте подсказкам рядом с выпадающими полями.

Запустите *Ячейку* ***№16*** **только один раз** для первого суда и, соответственно, первого ряда в таблице. **Чтобы занести новую информацию** о других судах и тем самым добавить новые ряды в таблицу, **используйте *Ячейку №17***.

In [ ]:
# Ячейка №16

list_with_column_names = ['Суд', '0 дней', '1-7 дней', '8+дн. - месяц', 'Больше месяца', 'Максимум +', 'Минимум -', 
                          '0> Будущее', 'Нет даты события', 'Нет даты размещения', 'Нет даты (сумма)', 'Всего записей', 
                          '0 дней %', '1-7 дней %', '8+дн. - месяц %', 'Больше месяца %', '0> Будущее %', 
                          'Нет даты события %', 'Нет даты размещения %', 'Нет даты (сумма) %']

add_a_court = Courts().create_a_row ()

final_comparative_df = pd.DataFrame ([list_with_column_names, add_a_court])
final_comparative_df.columns = final_comparative_df.iloc[0]
final_comparative_df = final_comparative_df.drop(0).reset_index().drop('index', axis = 1)
final_comparative_df

**Запустите** *Ячейку* ***№17***, чтобы **добавить новую информацию** о других судах **и** тем самым добавить **новые ряды в таблицу**.

Всё точно так же, как при запуске *Ячейки №16*. Следуйте подсказкам рядом с выпадающими полями.

Когда закончите работу с программой, обязательно сохраните получившуюся таблицу, запустив *Ячейку №18*.

In [ ]:
# Ячейка №17

add_a_court = Courts().create_a_row ()
add_a_row = pd.Series(add_a_court, index = final_comparative_df.columns)
final_comparative_df = final_comparative_df.append (add_a_row, ignore_index=True)
final_comparative_df

**Запустите** *Ячейку №18*, чтобы **сохранить файл** под названием "Сравнение судов.xlsx".

In [ ]:
# Ячейка №18

final_comparative_df.to_excel (f"Сравнение судов.xlsx")
print ('Выгрузка прошла успешно! :)')

## Общая информация о программе ##

Автор программы: Мария Казакова, Москва, 2020 год

Контакты: marikasakowa@gmail.com